# Create a deployment

## Setup

### Import libraries

In [27]:
import time
import datetime
from datetime import timedelta
from os.path import join

import datarobot as dr
import numpy as np
import pandas as pd
import helper

### Retrieve relevant projects

In [28]:
projects_df = pd.read_pickle("results/VolPred_percentage_each_1min_v_2022-11-14-23-12-28.pkl")  


In [29]:
projects_df.head()

,project_id,slice,url,project
0,6372796b52dd086a82a4337c,15:31,https://app.datarobot.com/projects/6372796b52d...,Project(VolPred_percentage_each_1min_15:31_v_2...
1,63727a1552dd086a82a43383,15:32,https://app.datarobot.com/projects/63727a1552d...,Project(VolPred_percentage_each_1min_15:32_v_2...
2,63727abc9ae8e86e8b8942e4,15:33,https://app.datarobot.com/projects/63727abc9ae...,Project(VolPred_percentage_each_1min_15:33_v_2...
3,63727b6b9ae8e86e8b8942ff,15:34,https://app.datarobot.com/projects/63727b6b9ae...,Project(VolPred_percentage_each_1min_15:34_v_2...
4,63727c1b2331f284bfa432cd,15:35,https://app.datarobot.com/projects/63727c1b233...,Project(VolPred_percentage_each_1min_15:35_v_2...


### Connect to DataRobot

In [30]:
# Instantiate the DataRobot connection

DATAROBOT_API_TOKEN = "" # Get this from the Developer Tools page in the DataRobot UI
# Endpoint - This notebook uses the default endpoint for DataRobot Managed AI Cloud (US)
DATAROBOT_ENDPOINT = "https://app.datarobot.com/api/v2" # This should be the URL you use to access the DataRobot UI

client = dr.Client(
    token=DATAROBOT_API_TOKEN, 
    endpoint=DATAROBOT_ENDPOINT,
    user_agent_suffix='' #Optional but helps DataRobot improve this workflow
)

dr.client._global_client = client

Connection to DataRobot established successfully


In [31]:
today = datetime.datetime.now().strftime('%Y-%m-%d')
today

'2022-11-14'

## Create a deployment for each top model

The goal of this notebook is very simple: you will loop through all of the projects you have created (find them in the "projects dataframe") and deploy the top-performing model on the Leaderboard. Then, you will update the "projects dataframe", which you can think of as a controller file, with the corresponding deployment ID, model ID, and model peformance (this example uses MASE as the metric).

In [44]:
models = []
MASE = []
deployments = []
prediction_server = dr.PredictionServer.list()[0]

for i, row in projects_df.iterrows():
    # print(str(i) + '-th job')
    # Load the project and get the recommended model
    project = dr.Project.get(row.project_id)
    recommended_model = project.recommended_model()
    
    parent = dr.Model.get(project.id,recommended_model.parent_model_id)
    metric = parent.metrics['MASE']['validation']
    
    name = project.project_name
    
    deployment = dr.Deployment.create_from_learning_model(recommended_model.id, label=name, description='Volume Prediction',default_prediction_server_id=prediction_server.id)
    
    models.append(recommended_model.id)
    deployments.append(deployment.id)
    MASE.append(metric)
    

In [51]:
projects_df['model'] = models

In [45]:
projects_df['deployment_id'] = deployments

In [46]:
projects_df['MASE'] = MASE

In [58]:
projects_df.tail()

,project_id,slice,url,project,deployment_id,MASE,model
24,637289a99ae8e86e8b894486,15:55,https://app.datarobot.com/projects/637289a99ae...,Project(VolPred_percentage_each_1min_15:55_v_2...,6372cddc0e6843f9d9dd6eae,0.76610,63728b6d43879a86279e5285
25,63728a4f2331f284bfa43430,15:56,https://app.datarobot.com/projects/63728a4f233...,Project(VolPred_percentage_each_1min_15:56_v_2...,6372cde01791a5dc46e78d09,0.85318,63728bfbec704547accaf364
26,63728af7b712e8ad568944b9,15:57,https://app.datarobot.com/projects/63728af7b71...,Project(VolPred_percentage_each_1min_15:57_v_2...,6372cde65a87f42014e78fe0,0.57489,63728ca437dc143b6490865b
27,63728b9e21b7aeebde893a19,15:58,https://app.datarobot.com/projects/63728b9e21b...,Project(VolPred_percentage_each_1min_15:58_v_2...,6372cdea0e6843f9d9dd6ee7,0.58169,63728d212418c32684f66898
28,63728c4c21b7aeebde893a25,15:59,https://app.datarobot.com/projects/63728c4c21b...,Project(VolPred_percentage_each_1min_15:59_v_2...,6372cdef29a7a77a961c237e,0.60961,63728e0f12d23f48da94c3e5


In [57]:
projects_df.to_pickle('results/VolPred_percentage_each_1min_v_2022-11-14-23-12-28.pkl') 

In [59]:
projects_df.model[0]

'63727b232bfd63533d45ffb5'

In [62]:
test = dr.Model.get(projects_df.project_id[0], projects_df.model[0])

In [63]:
test

DatetimeModel('RandomForest Regressor')